In [1]:
import pandas as pd
import csv
import langid
import numpy as np
import requests
import json
import re
import string
import nltk
import unicodedata
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import defaultdict

In [2]:
# download the Indonesian stopwords from NLTK
nltk.download('stopwords')
stop_words = set(nltk.corpus.stopwords.words('indonesian'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\irfan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# Load data
data = pd.read_csv('Bahan/gabung_all.csv', encoding='utf-8')

In [4]:
# Membuat kamus kata baku dan tidak baku
baku = defaultdict(set)
with open("Bahan/kata_dasar_kbbi.csv", "r", encoding="utf-8") as file:
    reader = csv.reader(file)
    next(reader)  # Skip header
    for row in reader:
        kata_baku = row[0]
        kategori = row[1]
        baku[kata_baku].add(kategori)

In [5]:
# Cek apakah 'case_folding' ada dalam dataframe
if 'text' in data.columns:
    # Ambil kolom 'case_folding' dari dataframe
    kolom_case_folding = data['text']
    
    # Fungsi untuk memisahkan hashtag
    def pisahkan_hashtag(kalimat):
        # Menggunakan regex untuk mencocokkan pola hashtag
        pola = r'#[A-Za-z0-9_]+'
        hasil = re.findall(pola, kalimat)
        
        # Mengganti hashtag dengan kata terpisah
        for hashtag in hasil:
            kata_terpisah = re.sub(r'([a-z])([A-Z])', r'\1 \2', hashtag)
            kalimat = kalimat.replace(hashtag, kata_terpisah)
        
        return kalimat

    # Memisahkan hashtag pada setiap kalimat dalam kolom 'case_folding'
    data['text_clean'] = kolom_case_folding.apply(pisahkan_hashtag)
    
    # Cetak dataframe untuk memverifikasi hasilnya
    print(data)
else:
    print("Kolom 'case_folding' tidak ada dalam dataframe.")

                                                    text     label  \
0      Lho!!!! Dia Yg Minta IKN , Kok Otorita. Di Beb...  negative   
1      Belum jadi kok sudah mau memindahkan ASN Pak? ...  negative   
2      Sampai di IKN, asn nya mengeluh...oala disini ...  negative   
3      Emang ikn jadi kapan? Kayak dia masih menjabat...  negative   
4      Tdk semudah itu ASN wajib pindah ke IKN tahun ...  negative   
...                                                  ...       ...   
12515  Kadang gimana gitu y pak, \nJadi ngerasa bener...  positive   
12516  DPR secara institusi sudah menyetujui IKN yg t...  positive   
12517  #2022JokowiSelesai Ikn,Nusantara,Bp \n@jokowi\...  positive   
12518  Sejarah besar ....\nBalap motor GP Mandalika\n...  positive   
12519  Bapak, Saya merantau ke Jakarta dr Jateng thn ...  positive   

                                              text_clean  
0      Lho!!!! Dia Yg Minta IKN , Kok Otorita. Di Beb...  
1      Belum jadi kok sudah mau memindahk

In [6]:
# Menyimpan hasil identifikasi ke dalam file CSV
data.to_csv("Hasil/hashtag.csv", index=False)

In [7]:
# Import corpus dari nltk
from nltk.tokenize import word_tokenize

# Fungsi untuk menghapus tanda baca menggunakan regex
def hapus_tanda_baca(kata):
    return re.sub(r'[^\w\s-]', '', kata)

# Fungsi untuk melakukan preprocessing pada kolom teks
def preprocess_text(text):

    # Menghapus karakter non-ASCII
    text = remove_non_ascii(text)

    # Remove URLs, mentions, and hashtags
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'\@\w+|\#', '', text)

    # Tokenisasi kalimat menjadi kata-kata
    kata_kata = word_tokenize(text)

    # Menghapus tanda baca dari setiap kata
    kata_kata_tanpa_tanda_baca = [hapus_tanda_baca(kata) for kata in kata_kata]

    # Menggabungkan kata-kata kembali menjadi kalimat
    kalimat_tanpa_tanda_baca = ' '.join(kata_kata_tanpa_tanda_baca)

    # Menggantikan tanda baca dengan spasi tunggal menggunakan regex
    kalimat_final = re.sub(r'(?<=[^\w\s-])', ' ', kalimat_tanpa_tanda_baca)

    # Menggantikan multiple spasi dengan satu spasi menggunakan regex
    kalimat_tanpa_spasi_ganda = re.sub(r'\s+', ' ', kalimat_final)

    # Remove special characters and digits
    kalimat_tanpa_tanda_baca_dan_digit = re.sub(r'[^\w\s-]', '', kalimat_tanpa_spasi_ganda)
    kalimat_tanpa_digit = re.sub(r'\d+', '', kalimat_tanpa_tanda_baca_dan_digit)

    # Convert to lowercase
    kalimat_final = kalimat_tanpa_digit.lower()

    # Remove whitespace leading & trailing
    kalimat_final = kalimat_final.strip()

    # Remove single char
    kalimat_final = re.sub(r"\b[a-zA-Z]\b", "", kalimat_final)

    return kalimat_final

def remove_non_ascii(text):
    return ''.join(c for c in unicodedata.normalize('NFKD', text) if unicodedata.category(c) != 'Mn')

# Melakukan preprocessing pada kolom teks
data['text_clean'] = data['text_clean'].apply(preprocess_text)

In [8]:
data

,text,label,text_clean
0,"Lho!!!! Dia Yg Minta IKN , Kok Otorita. Di Beb...",negative,lho dia yg minta ikn kok otorita di bebani ya ...
1,Belum jadi kok sudah mau memindahkan ASN Pak? ...,negative,belum jadi kok sudah mau memindahkan asn pak y...
2,"Sampai di IKN, asn nya mengeluh...oala disini ...",negative,sampai di ikn asn nya mengeluh oala disini bar...
3,Emang ikn jadi kapan? Kayak dia masih menjabat...,negative,emang ikn jadi kapan kayak dia masih menjabat ...
4,Tdk semudah itu ASN wajib pindah ke IKN tahun ...,negative,tdk semudah itu asn wajib pindah ke ikn tahun ...
...,...,...,...
12515,"Kadang gimana gitu y pak, \nJadi ngerasa bener...",positive,kadang gimana gitu pak jadi ngerasa bener bui...
12516,DPR secara institusi sudah menyetujui IKN yg t...,positive,dpr secara institusi sudah menyetujui ikn yg t...
12517,"#2022JokowiSelesai Ikn,Nusantara,Bp \n@jokowi\...",positive,jokowi selesai ikn nusantara bp akan sangat ma...
12518,Sejarah besar ....\nBalap motor GP Mandalika\n...,positive,sejarah besar balap motor gp mandalika terbent...


In [9]:
# Menyimpan hasil identifikasi ke dalam file CSV
data.to_csv("Hasil/data_gabung_all.csv", index=False)

In [10]:
# Membuat list untuk menyimpan data yang tidak baku
data_tidak_baku = []

In [12]:
from collections import Counter

# Melakukan identifikasi kata tidak baku
data_tidak_baku = []
for i, tweet in data.iterrows():
    text = tweet["text_clean"]
    tokens = text.split()
    for j, token in enumerate(tokens):
        if token.lower() not in baku:
            data_tidak_baku.append({
                "text": token,
                "edit": "",
            })

# Menghitung kata-kata tidak baku yang sering muncul
counter = Counter(token["text"].lower() for token in data_tidak_baku)
kata_tidak_baku_serimg_muncul = counter.most_common()  # Mengambil semua kata yang sering muncul

# Menampilkan kata-kata tidak baku yang sering muncul
print("Kata-kata tidak baku yang sering muncul:")
for kata, frekuensi in kata_tidak_baku_serimg_muncul:
    print(f"{kata}: {frekuensi}")

# Membuat DataFrame dari kata-kata tidak baku yang sering muncul
df_kata_tidak_baku = pd.DataFrame(kata_tidak_baku_serimg_muncul, columns=["Kata Tidak Baku", "Frekuensi"])

# Menyimpan DataFrame ke dalam file CSV
df_kata_tidak_baku.to_csv("04. Jumlah Kata Sering Muncul/jumlah_kata_tidak_baku.csv", index=False)

Kata-kata tidak baku yang sering muncul:
ikn: 13310
yg: 3920
gak: 1143
ga: 911
nya: 896
pemilu: 807
jokowi: 793
pembangunan: 680
utk: 523
tdk: 493
semoga: 453
dgn: 442
jd: 411
kalimantan: 405
sdh: 404
udah: 386
klo: 373
dr: 355
pemerintah: 350
dulu: 342
jakarta: 305
dg: 297
krn: 296
softbank: 284
migor: 276
ibukota: 264
setuju: 255
gk: 254
jg: 248
tuh: 248
mangkrak: 247
tp: 245
menjadi: 241
jawa: 240
jabatan: 224
esemka: 224
uu: 223
lg: 220
pake: 211
sebagai: 209
apbn: 204
-: 203
otorita: 202
membangun: 202
nggak: 195
alasan: 192
jgn: 181
setelah: 175
gitu: 171
anggaran: 169
lu: 168
blm: 167
negeri: 167
pemindahan: 166
ditunda: 166
penundaan: 157
org: 157
berkemah: 156
utang: 156
hutang: 155
katanya: 153
dlm: 152
ikn_ikatan: 148
pejabat: 146
allah: 145
ngotot: 144
gimana: 143
pemimpin: 141
dibangun: 139
diabaikan: 136
iknrakyat: 133
sy: 132
dll: 131
urusan: 128
ngurus: 126
gw: 126
disana: 124
sampe: 124
luhut: 123
ternyata: 119
kl: 119
trus: 119
terjadi: 118
bener: 117
mikir: 115
asn: 

In [13]:
# Menyimpan data yang tidak baku ke dalam file CSV baru
df_tidak_baku = pd.DataFrame(data_tidak_baku, columns=["text"])
df_tidak_baku.to_csv("03. Daftar Kata baku dan tidak baku/kumpulan_kata_tidak_baku.txt", sep="\t", index=False)